In [ ]:
import boto3
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO
from shapely.geometry import Polygon, MultiPolygon

In [ ]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [ ]:
from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

### Resumo da análise

- Existem colunas duplicadas que representam a mesma variável [``start_season``, ``start_of_season``], além de [``end_season``, ``end_of_season``], [``peak_of_season``, ``peak_season``]
- Existem dados duplicados considerando o ``eopatch_location``
- Não existe mais a coluna ``id_talhao``?

### Global Variables

In [ ]:
config = yaml_utils.load_config("/agrilearn_app/output/config/experiment_12.yaml")

In [ ]:
datasets = config['evaluate']

In [ ]:
report_datasets = evaluate_datasets(datasets)

In [ ]:
config["evaluate"][1]

In [ ]:
import os
import geopandas as gpd

def evaluate_all_datasets(datasets):
    """
    Avalia uma lista de datasets realizando as seguintes operações:
    - Lê o geopackage_test_data_path e conta o tamanho dos dados.
    - Verifica e conta a quantidade de arquivos na pasta eopatch_folder.
    - Verifica se as colunas label_true e label_eopatch_location existem no geopackage_test_data_path.
    
    Args:
        datasets (list): Lista de dicionários contendo informações dos datasets.
        
    Returns:
        pd.DataFrame: DataFrame contendo o relatório final por dataset.
    """
    results = []
    
    for dataset in datasets:
        result = {}
        result['dataset_name'] = dataset['dataset_name']
        
        # Lê o arquivo geopackage e conta o tamanho dos dados
        try:
            gdf = gpd.read_file(dataset['geopackage_test_data_path'])
            result['data_size'] = len(gdf)
        except Exception as e:
            result['data_size'] = f"Error: {e}"
        
        # Conta a quantidade de arquivos na pasta eopatch
        try:
            result['eopatch_file_count'] = len(os.listdir(dataset['eopatch_folder']))
        except Exception as e:
            result['eopatch_file_count'] = f"Error: {e}"
        
        # Verifica se as colunas label_true e label_eopatch_location existem no arquivo geopackage
        try:
            result['label_true_exists'] = dataset['label_true'] in gdf.columns
            result['label_eopatch_location_exists'] = dataset['label_eopatch_location'] in gdf.columns
        except Exception as e:
            result['label_true_exists'] = f"Error: {e}"
            result['label_eopatch_location_exists'] = f"Error: {e}"
        
        results.append(result)
    
    # Cria um DataFrame com os resultados
    df_results = pd.DataFrame(results)
    
    return df_results

# 1. Read Datasets

### Data description
- monitoring_class: é o ``conjunto`` que representa a cultura de interesse do cliente ou da amostra de treinamento (SOYBEAN, CORN)
- period: é o ``texto`` que representa a safra agricola (2023/2024, 2024/2025)
- fonte: é o ``texto`` da origem dos dados (mapas temáticos, banco de dados)
- state: é o ``texto`` do estado do polígono
- area: é o ``ponto flutuante`` da área em métros
- micro: é o ``inteiro`` que representa a micro região
- start_season: é o ``timestamp`` que inicia a safra pra determinada cultura (monitoring_class) e safra (period)
- end_season: é o  ``timestamp`` que termina a safra pra determinada cultura (monitoring_class) e safra (period)
- peak_start: é o ``timestamp`` que inicia o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period)
- peak_end: é o ``timestamp`` que termina o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period) (Obs: temos um calendário pra isso, um arquivo yaml)
- eopath_location: é o ``texto`` que representa o caminho onde o eopatch está salvo (imagens p/ inferência)
- start_of_season: é o ``timestamp`` da data da emergencia da cultura
- peak_of_season: é o ``timestamp`` da data do pico vegetativo da cultura
- end_of_season: é o ``timestamp`` da data da colheita da cultura
- length_of_season: é o ``inteiro`` com a duração do cultivo em dias
- is_valid_metrics:
- sos_valid:
- pos_valid:
- eos_valid:
- los_valid:
- planting_start: é o ``timestamp`` da data do inicio do plantio.
- planting_end: é o ``timestamp`` da data do fim do plantio.
- start_of_cycle: é o ``timestamp`` da data do inio do cultivo segundo o calendário agrícola
- end_of_cycle: é o ``timestamp`` da data do fim do caledário de cultivo segundo calendário agrícola
- length_of_cycle: é o ``inteiro`` que representa o tamanho do ciclo em dias
- is_valid:
- is_valid_POS:
- is_valid_LOS:
- set_type: é o ``conjunto`` que representar a divisão do dado entre treino, validação e teste
- sampled_date: é o ``texto`` com a data do sample em mês e ano
- cultura_2:
- geometry: é a ``geometria`` do polígono 

In [ ]:
gdfs = []

# Lê cada Geopackage e adiciona o GeoDataFrame à lista
for path in GEOPACKAGE_PATH:
    gdf = gpd.read_file(path)
    gdf['dataset_source'] = path  # Adiciona a coluna de origem
    gdfs.append(gdf)

# Concatena todos os GeoDataFrames em um único GeoDataFrame
if gdfs:
    df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    print(f"Dataset final possui {df.shape[0]} linhas")
    df.head()
else:
    print("Nenhum GeoDataFrame válido encontrado.")

In [ ]:
df['dataset_source'].value_counts()

In [ ]:
df['monitoring_class'].value_counts()

In [ ]:
df.head()

In [ ]:
df['class'].value_counts()

In [ ]:
df['eopath_location'].value_counts()

In [ ]:
df.columns

In [ ]:
df['start_of_season'].sample()

In [ ]:
datetime_columns = ['start_season', 
                    'end_season', 
                    'peak_start', 
                    'peak_end',
                    'start_of_season',
                    'peak_of_season',
                    'end_of_season',
                    'planting_start',
                    'planting_end',
                    'start_of_cycle',
                    'end_of_cycle'                  
                   ]

    # Verifica se a coluna existe, se existir formata, senão printa mensagem
for col in datetime_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
        except Exception as e:
            print(f"Erro ao formatar a coluna {col}: {e}")
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")

# Exibe informações das colunas datetime
df[datetime_columns].info()

In [ ]:
df['monitoring_class'].value_counts()

In [ ]:
df['state'].value_counts()

In [ ]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

In [ ]:
df[datetime_columns].info()

In [ ]:
print(f"Dados de {df['start_season'].min()} a {df['end_season'].max()}")

In [ ]:
list(df.columns)

# 02. Data Integration in columns

In [ ]:
df[['start_season', 'start_of_season']].sample(10)

In [ ]:
df[['length_of_season', 'LOS']].sample(10)

In [ ]:
df[['end_season', 'end_of_season']].sample(10)

In [ ]:
df[['eopath_location', 'sentinel_eopatch_current']].sample(10)

In [ ]:
# # Atribui a coluna principal que são nulas os dados da coluna que tem nome diferente no dataset
# df['length_of_season'] = df['length_of_season'].fillna(df['LOS'])
# df['start_season'] = df['start_season'].fillna(df['start_of_season'])
# df['end_season'] = df['end_season'].fillna(df['end_of_season'])
# df['eopath_location'] = df['eopath_location'].fillna(df['sentinel_eopatch_current'])

# 03. Check Data Quality

## 3.1 Drop Columns NaN

In [ ]:
shape_before = df.shape[1]
df.dropna(axis=1, how='all', inplace=True)
print(f"Removed Columns: {shape_before - df.shape[1]}, Percentage: {(shape_before - df.shape[1]) / shape_before * 100:.2f}%")

## 3.2 Check NaN Values

In [ ]:
# Calcula a quantidade de registros nulos e a porcentagem de registros nulos
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)

In [ ]:
columns_to_delete = list(df_check_NaN[df_check_NaN['NaN percentage'] > 50].index)
df.drop(columns=columns_to_delete, inplace=True)

## 3.3 Transform all Geometry to Multipolygon

In [ ]:
df['geometry_type'] = df['geometry'].apply(lambda geom: 'Polygon' if isinstance(geom, Polygon) else 'MultiPolygon')
df['geometry_type'].value_counts()

In [ ]:
df['geometry'] = df['geometry'].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)

In [ ]:
df['geometry_type'] = df['geometry'].apply(lambda geom: 'Polygon' if isinstance(geom, Polygon) else 'MultiPolygon')
df['geometry_type'].value_counts()

## 3.4 Check Duplicated

In [ ]:
df_check_duplicated = df[df.duplicated(subset=['geometry', 'period'], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby(['geometry', 'period']).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

In [ ]:
# df_check_duplicated.to_csv('/agrilearn_app/datasets/base/reports/base_all_06-03-2024.csv',
#                            sep=';',
#                            decimal=',',
#                            float_format='%.4f')

In [ ]:
df['dataset_source'].value_counts()

In [ ]:
shape_before = df.shape[0]
df.drop_duplicates(subset=['geometry', 'period'], inplace=True)
print(f"Removed records: {shape_before - df.shape[0]}, Percentage: {(shape_before - df.shape[0]) / shape_before * 100:.2f}%")

### Update NaN after DROP

In [ ]:
# Update NaN values
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)

# 04. Feature Engineering

In [ ]:
df['safra'] = df['period'].apply(lambda x: 'safrinha' if x.split('/')[0] == x.split('/')[1] else 'safra')

# 05. Univariate Data Analysis

### 5.1 Analysing Target y ``monitoring_class``

In [ ]:
df_check_NaN.loc['monitoring_class']

In [ ]:
df['monitoring_class'].nunique()

In [ ]:
pd.concat([df['monitoring_class'].value_counts(),
          df['monitoring_class'].value_counts(normalize=True)*100], axis=1)

### 5.2 Analysing dados por ``estado``

In [ ]:
df_check_NaN.loc['state']

In [ ]:
df['state'].nunique()

In [ ]:
pd.concat([df['state'].value_counts(),
          df['state'].value_counts(normalize=True)*100], axis=1)

### 5.3 Analysis ``period``

In [ ]:
df_check_NaN.loc['period']

In [ ]:
df['period'].nunique()

In [ ]:
pd.concat([df['period'].value_counts(),
          df['period'].value_counts(normalize=True)*100], axis=1)

### 5.4 Analysis ``micro``

In [ ]:
df_check_NaN.loc['micro']

In [ ]:
df['micro'].nunique()

In [ ]:
pd.concat([df['micro'].value_counts(),
          df['micro'].value_counts(normalize=True)*100], axis=1)

### 5.5 Analysis ``sampled_date``

In [ ]:
df_check_NaN.loc['sampled_date']

In [ ]:
df['sampled_date'].nunique()

In [ ]:
pd.concat([df['sampled_date'].value_counts(),
          df['sampled_date'].value_counts(normalize=True)*100], axis=1)

### 5.6 Analysis ``set_type``

In [ ]:
df_check_NaN.loc['set_type']

In [ ]:
pd.concat([df['set_type'].value_counts(),
          df['set_type'].value_counts(normalize=True)*100], axis=1)

### 5.7 Analysis of ``length_of_season (LOS)``

In [ ]:
df_check_NaN.loc['length_of_season']

In [ ]:
df['length_of_season'] = df['length_of_season'].astype(float)

In [ ]:
df['length_of_season'].describe()

In [ ]:
pd.concat([df['length_of_season'].value_counts(),
          df['length_of_season'].value_counts(normalize=True)*100], axis=1)

In [ ]:
df[df['length_of_season'] < 100]

### 5.8 Analysis of ``area``

In [ ]:
df_check_NaN.loc['area']

In [ ]:
pd.concat([df['area'].value_counts(),
          df['area'].value_counts(normalize=True)*100], axis=1)

### 5.9 Analysis of ``length_of_cycle (LOC)``

In [ ]:
df_check_NaN.loc['length_of_cycle']

In [ ]:
df['length_of_cycle'] = df['length_of_cycle'].astype(float)

In [ ]:
df['length_of_cycle'].describe()

In [ ]:
pd.concat([df['length_of_cycle'].value_counts(),
          df['length_of_cycle'].value_counts(normalize=True)*100], axis=1)

### 5.10 Analysis of ``fonte``

In [ ]:
df_check_NaN.loc['fonte']

In [ ]:
pd.concat([df['fonte'].value_counts(),
          df['fonte'].value_counts(normalize=True)*100], axis=1)

In [ ]:
### 5.11 Analysis of ``eopath_location``

In [ ]:
df_check_NaN.loc['eopath_location']

In [ ]:
df['eopath_location'].nunique()

In [ ]:
df.shape[0]

In [ ]:
df_check_duplicated = df[df.duplicated(subset=[label_eopatch_path], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby([label_eopatch_path]).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

In [ ]:
# df_check_duplicated.to_csv('/agrilearn_app/datasets/base/reports/base_all_by_eopatch_location_11-03-2024.csv',
#                            sep=';',
#                            decimal=',',
#                            float_format='%.4f')

In [ ]:
# Verifica se há duplicatas na coluna especificada
duplicates = df['eopath_location'].duplicated(keep=False)

# Calcula a quantidade de dados duplicados e o percentual em relação ao dataset final
total_rows = len(df)
duplicate_count = duplicates.sum()
duplicate_percentage = (duplicate_count / total_rows) * 100

print(f"Quantidade de dados duplicados: {duplicate_count}")
print(f"Percentual de dados duplicados: {duplicate_percentage:.2f}%")

* **Question**: O mesmo geopackage está apontando par ao mesmo eopatch_location? *

### 5.12 Check ``start_season`` and ``end_season``

In [ ]:
df_check_NaN.loc[['start_season', 'end_season']]

In [ ]:
df[['start_season', 'end_season']].describe()

In [ ]:
df.groupby(df['start_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['end_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['start_season'] == df['end_season']]

### 5.13 Check ``start_season`` and ``end_season``

In [ ]:
df_check_NaN.loc[['peak_start', 'peak_end']]

In [ ]:
df[['peak_start', 'peak_end']].describe()

In [ ]:
df[['peak_start', 'peak_end']].describe()

In [ ]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['peak_end'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['peak_start'] == df['peak_end']]

In [ ]:
df['peak_start'].dt.year.unique()

In [ ]:
df['peak_end'].dt.year.unique()

In [ ]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

### 5.14 Check ``planting_start`` and ``planting_end``

In [ ]:
df_check_NaN.loc[['planting_start', 'planting_end']]

In [ ]:
df[['planting_start', 'planting_end']].describe()

In [ ]:
df.groupby(df['planting_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df.groupby(df['planting_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

In [ ]:
df[df['planting_start'] == df['planting_end']]

In [ ]:
df['planting_start'].dt.year.unique()

In [ ]:
df['planting_end'].dt.year.unique()

# 6. Multivariate Analysis

In [ ]:
columns_to_group = ['safra', 'state', 'monitoring_class']
report_distribuition = df.groupby(columns_to_group).agg(count=(label_eopatch_path, 'count')).reset_index()

# Calcula o percentual em relação ao total
total_count = len(df)
report_distribuition['percentage'] = (report_distribuition['count'] / total_count) * 100

In [ ]:
report_distribuition

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in df['monitoring_class'].value_counts().items()])
filename=f'/agrilearn_app/datasets/base/reports/check_distribuition_by_class_and_state_{result_string}.csv'

In [ ]:
report_distribuition.to_csv(filename,
    sep=';',
    decimal=',',
    float_format='%.4f',
)
filename

# 7. Data Integration to Endafoclimatiocas region

In [ ]:
from agrilearn.crop_classification import geom_utils

In [ ]:
GEOPACKAGE_ENDO_SOJA = ["/agrilearn_app/datasets/other/meso-soja/ref_edf_soja_processed.gpkg"]

In [ ]:
gdfs_edf = []

# Lê cada Geopackage e adiciona o GeoDataFrame à lista
for path in GEOPACKAGE_ENDO_SOJA:
    gdf = gpd.read_file(path)
    gdf['dataset_source'] = path  # Adiciona a coluna de origem
    gdfs_edf.append(gdf)

# Concatena todos os GeoDataFrames em um único GeoDataFrame
if gdfs:
    df_edf = gpd.GeoDataFrame(pd.concat(gdfs_edf, ignore_index=True))
    print(f"Dataset final possui {df_edf.shape[0]} linhas")
    df_edf.head()
else:
    print("Nenhum GeoDataFrame válido encontrado.")

In [ ]:
df = geom_utils.add_edf_feature(df_target=df,
                                df_edf=df_edf, 
                                label_edf="edf")

In [ ]:
columns_to_group = ['safra', 'edf', 'monitoring_class']
report_distribuition = df.groupby(columns_to_group).agg(count=(label_eopatch_path, 'count')).reset_index()

# Calcula o percentual em relação ao total
total_count = len(df)
report_distribuition['percentage'] = (report_distribuition['count'] / total_count) * 100

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in df['monitoring_class'].value_counts().items()])
result_string

In [ ]:
# report_distribuition.to_csv(f'/agrilearn_app/datasets/base/reports/check_distribuition_by_class_and_edf_{result_string}.csv',
#     sep=';',
#     decimal=',',
#     float_format='%.4f',
# )

In [ ]:
result_string

# 8. Train and Test Split

In [ ]:
from agrilearn.crop_classification.training_utils import split_dataset_by_state_and_strafity, split_dataset_with_sample_restriction

In [ ]:
stratify_labels=['safra', 'edf', 'monitoring_class']

In [ ]:
train_set, test_set = split_dataset_with_sample_restriction(df,
                                                            stratify_labels,
                                                            threshold=1000,
                                                            small_sample_train_size=0.5,
                                                            large_sample_train_size=0.5)

In [ ]:
train_set['monitoring_class'].value_counts()

In [ ]:
test_set['monitoring_class'].value_counts()

In [ ]:
test_set.groupby(stratify_labels).agg(count=('period', 'count')).query("count > 500").shape

In [ ]:
test_set.groupby(stratify_labels).agg(count=('period', 'count'))

In [ ]:
test_set.groupby(stratify_labels).agg(count=('period', 'count'))

In [ ]:
test_set.groupby(stratify_labels).agg(count=('period', 'count')).query("count > 1000").shape

In [ ]:
test_set.groupby(stratify_labels).agg(count=('period', 'count')).query("count > 500").shape

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in train_set['monitoring_class'].value_counts().items()])
filename_train=f"{OUTPUT_DIR}/{result_string}_train.gpkg"
filename_train

In [ ]:
train_set['dataset_part']='train'
train_set.to_file(filename_train, driver='GPKG', engine='fiona')
filename_train

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in test_set['monitoring_class'].value_counts().items()])
filename_test= f"{OUTPUT_DIR}/{result_string}_test.gpkg"
filename_test

In [ ]:
test_set['dataset_part']='test'
test_set.to_file(filename_test, driver='GPKG', engine='fiona')
filename_test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def split_stratifield_train_test_split(df_, stratify_columns, train_size=0.7, min_instances=2, random_state=42):
    """
    Split the dataset into training and testing sets using stratification.

    Parameters
    ----------
    df_ : pd.DataFrame
        DataFrame containing the data.
    stratify_labels : list
        List of columns to be used for stratification.
    train_size : float, optional
        Proportion of the dataset to be used for training (default is 0.7).
    random_state : int, optional
        Random state for reproducibility (default is 42).

    Returns
    -------
    tuple
        DataFrames for training and testing.
    """
    df_train_list = []
    df_test_list = []
    
    for category, group in df_.groupby(stratify_labels):
        if len(group) < min_instances:
            df_train_list.append(group)
            df_test_list.append(group)
        else:
            train, test = train_test_split(group,
                                           train_size=train_size, 
                                           random_state=random_state,
                                           stratify=group[stratify_labels])
            df_train_list.append(train)
            df_test_list.append(test)
    df_train = pd.concat(df_train_list).reset_index(drop=True)
    df_test = pd.concat(df_test_list).reset_index(drop=True)
                                           

    return df_train, df_test

In [ ]:
df_train, df_val = split_stratifield_train_test_split(train_set,
                            stratify_labels,
                            train_size=0.8,
                            random_state=42)

In [ ]:
df['monitoring_class'].value_counts()

In [ ]:
df_train['monitoring_class'].value_counts()

In [ ]:
df_val['monitoring_class'].value_counts()

In [ ]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in df_val['monitoring_class'].value_counts().items()])
filename_val= f"{OUTPUT_DIR}/{result_string}_val.gpkg"
filename_val

In [ ]:
df_val['dataset_part']='val'
df_val.to_file(filename_val, driver='GPKG', engine='fiona')
filename_val